In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/phonereviews/data.csv


In [2]:
data = pd.read_csv('/kaggle/input/phonereviews/data.csv')
data = data.dropna()
data_filtered = data.loc[data['Rating'].isin({1, 2, 4, 5})]
df = data_filtered
df = df.reset_index(drop=True)
df.loc[(df['Rating'] >= 4), 'Sentiment'] = 'positive'
df.loc[(df['Rating'] <= 2), 'Sentiment'] = 'negative'
df = df.loc[df.apply(lambda row: len(row['Review']) >= 20, axis=1)]
df['Sentiment'].value_counts()

positive    326183
negative     67920
Name: Sentiment, dtype: int64

In [3]:
from sklearn.model_selection import train_test_split

texts = df['Review'].values
labels = df['Sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(
    texts,
    labels,
    test_size=0.1,
    random_state=1,
)
print(f'len(X_train) = {len(X_train)}')
print(f'len(y_train) = {len(y_train)}')
print(f'len(X_test) = {len(X_test)}')
print(f'len(y_test) = {len(y_test)}')

len(X_train) = 354692
len(y_train) = 354692
len(X_test) = 39411
len(y_test) = 39411


In [4]:
X_train[:5]

array(['За такие деньги это хороший телефон',
       'Купил недавно, но не могу нарадоваться! Раньше была Sony Xperia neo - очень плохо ловила wi-fi, да и вообще морально устарела. Выбирал себе новый телефон, наверное, полгода - очень много вариантов пересмотрел, раз по 10 каждый отвергал и потом заново рассматривал:). В итоге остановился на E975 и, что для меня редкость, ни капельки не пожалел! Т. е. вообще нет ничего, что раздражает или не нравится - сплошная гора позитивных эмоций! Я влюблён в этот телефон и боюсь только случайно его уронить (стеклянный, всё же).PS Носите без чехла (я хочу купить "кармашек", чтобы когда достаю телефон - в руках без всяких нагромождений было)! В руках держать очень приятно!P. S. Не понравилась только базовая клавиатура на кириллице, но она за 30 секунд меняется на нормальную от гугла, например:)P. P. S. UPD от 14.03.2016: Телефон всё ещё как новенький и я всё так же в него влюблён:)',
       'Отличный телефон, nokia рулит, сколько пользуюсь столько и

In [5]:
y_train[:5]

array(['positive', 'positive', 'positive', 'positive', 'positive'],
      dtype=object)

In [8]:
import csv

n_small = 50_000
y_train_mod = np.where(
    y_train == 'positive', 
    '__label__positive', 
    '__label__negative',
)
pd.DataFrame({'label': y_train_mod[:n_small], 'text': X_train[:n_small]}).to_csv(
    'train_small.txt',
    index=False,
    sep=' ',
    header=None,
    quoting=csv.QUOTE_NONE,
    quotechar='',
    escapechar=' ',
)

In [9]:
with open('train_small.txt', 'r', encoding='utf-8') as f:
    text = f.read(2048)
print(text)

__label__positive За  такие  деньги  это  хороший  телефон
__label__positive Купил  недавно,  но  не  могу  нарадоваться!  Раньше  была  Sony  Xperia  neo  -  очень  плохо  ловила  wi-fi,  да  и  вообще  морально  устарела.  Выбирал  себе  новый  телефон,  наверное,  полгода  -  очень  много  вариантов  пересмотрел,  раз  по  10  каждый  отвергал  и  потом  заново  рассматривал:).  В  итоге  остановился  на  E975  и,  что  для  меня  редкость,  ни  капельки  не  пожалел!  Т.  е.  вообще  нет  ничего,  что  раздражает  или  не  нравится  -  сплошная  гора  позитивных  эмоций!  Я  влюблён  в  этот  телефон  и  боюсь  только  случайно  его  уронить  (стеклянный,  всё  же).PS  Носите  без  чехла  (я  хочу  купить  "кармашек",  чтобы  когда  достаю  телефон  -  в  руках  без  всяких  нагромождений  было)!  В  руках  держать  очень  приятно!P.  S.  Не  понравилась  только  базовая  клавиатура  на  кириллице,  но  она  за  30  секунд  меняется  на  нормальную  от  гугла,  например:)P.  P.  S.

In [52]:
%%time
import fasttext

model = fasttext.train_supervised('train_small.txt', wordNgrams=2)

Read 3M words
Number of words:  266677
Number of labels: 2
Progress: 100.0% words/sec/thread: 1157925 lr:  0.000000 avg.loss:  0.205066 ETA:   0h 0m 0s


CPU times: user 14.9 s, sys: 900 ms, total: 15.8 s
Wall time: 6.6 s


In [56]:
y_test_mod = np.where(
    y_test == 'positive', 
    '__label__positive', 
    '__label__negative',
)
pd.DataFrame({'label': y_test_mod, 'text': X_test}).to_csv(
    'test.txt',
    index=False,
    sep=' ',
    header=None,
    quoting=csv.QUOTE_NONE,
    quotechar='',
    escapechar=' ',
)

In [55]:
print(model.predict('Хороший телефон'))
print(model.predict('Ужасный телефон'))

(('__label__positive',), array([1.00001001]))
(('__label__negative',), array([1.00001001]))


In [57]:
%%time
model.test('test.txt')

CPU times: user 1.42 s, sys: 25 ms, total: 1.45 s
Wall time: 1.43 s


(39411, 0.9065743066656518, 0.9065743066656518)

In [59]:
y_pred = []
for text in X_test:
    y_pred.append(model.predict(text)[0][0])
y_pred[:10]

['__label__positive',
 '__label__negative',
 '__label__positive',
 '__label__positive',
 '__label__positive',
 '__label__positive',
 '__label__positive',
 '__label__positive',
 '__label__positive',
 '__label__negative']

In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_test_mod, y_pred))

                   precision    recall  f1-score   support

__label__negative       0.82      0.58      0.68      6742
__label__positive       0.92      0.97      0.95     32669

         accuracy                           0.91     39411
        macro avg       0.87      0.78      0.81     39411
     weighted avg       0.90      0.91      0.90     39411



In [61]:
y_train_mod = np.where(
    y_train == 'positive', 
    '__label__positive', 
    '__label__negative',
)
pd.DataFrame({'label': y_train_mod, 'text': X_train}).to_csv(
    'train.txt',
    index=False,
    sep=' ',
    header=None,
    quoting=csv.QUOTE_NONE,
    quotechar='',
    escapechar=' ',
)

In [65]:
%%time
model = fasttext.train_supervised('train.txt', wordNgrams=2)

Read 21M words
Number of words:  769104
Number of labels: 2
Progress:  99.7% words/sec/thread: 1164444 lr:  0.000265 avg.loss:  0.095747 ETA:   0h 0m 0s1165484 lr:  0.008905 avg.loss:  0.103088 ETA:   0h 0m 2s

CPU times: user 1min 38s, sys: 2.07 s, total: 1min 41s
Wall time: 38.6 s


Progress: 100.0% words/sec/thread: 1163775 lr:  0.000000 avg.loss:  0.095540 ETA:   0h 0m 0s


In [66]:
y_pred = []
for text in X_test:
    y_pred.append(model.predict(text)[0][0])
print(classification_report(y_test_mod, y_pred))

                   precision    recall  f1-score   support

__label__negative       0.95      0.90      0.93      6742
__label__positive       0.98      0.99      0.99     32669

         accuracy                           0.98     39411
        macro avg       0.97      0.95      0.96     39411
     weighted avg       0.98      0.98      0.98     39411

